In [1]:
import numpy as np
import pandas as pd
import warnings
import gc
import tensorflow as tf
import holidays

from tensorflow.keras import layers
from tensorflow import keras

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
from sklearn.preprocessing import OneHotEncoder


import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


/kaggle/input/tabular-playground-series-mar-2022/sample_submission.csv
/kaggle/input/tabular-playground-series-mar-2022/train.csv
/kaggle/input/tabular-playground-series-mar-2022/test.csv


In [2]:
def split_sequences(sequences, n_steps):
	X, y = list(), list()
	for i in range(len(sequences)):
		# find the end of this pattern
		end_ix = i + n_steps
		# check if we are beyond the dataset
		if end_ix > len(sequences):
			break
		# gather input and output parts of the pattern
		seq_x, seq_y = sequences[i:end_ix, :-1], sequences[end_ix-1, -1]
		X.append(seq_x)
		y.append(seq_y)
	return np.array(X), np.array(y)

In [3]:
def label_encoder(df, column, one_hot_encoder=None):
    if one_hot_encoder is None:
        one_hot_encoder = OneHotEncoder()
        one_hot_transformed = one_hot_encoder.fit_transform(df[column].to_numpy().reshape(-1, 1)).toarray()
    else:
        one_hot_transformed = one_hot_encoder.transform(df[column].to_numpy().reshape(-1, 1)).toarray()
    one_hot_df = pd.DataFrame(one_hot_transformed, columns=one_hot_encoder.get_feature_names())
    df = pd.concat([df, one_hot_df], axis=1).drop([column], axis=1)
    
    return one_hot_encoder, df

In [4]:
def preprocess_dates(df, date_column='time'):
    df = df.copy()
    df[date_column] = pd.to_datetime(df[date_column])
    df['weekday'] = df[date_column].dt.weekday
    df['quarter'] = df[date_column].dt.quarter
    df['day_of_year'] = df[date_column].dt.day_of_year
    df['is_month_start'] = df[date_column].dt.is_month_start.astype("int8")
    df['is_month_end'] = df[date_column].dt.is_month_end.astype("int8")
    df['month'] = df[date_column].dt.month
    return df

In [5]:
def preprocess_holidays(df, date_column='time'):
    holiday_us = holidays.CountryHoliday(country='US', years=[1991])
    dates = list(holiday_us.keys())
    dates = sorted(pd.to_datetime(dates))
    df = df.copy()
    df['is_holiday'] = df[date_column].apply(lambda x : 1 if x in dates else 0)
    return df

In [6]:
def preprocess_timeseries(df):
    df = df.copy()
    df['sin_day_of_year'] = np.sin(df['day_of_year'])
    df['sin_month'] = np.sin(df['month'])
    return df

In [7]:
class Time2Vector(tf.keras.layers.Layer):
    def __init__(self, seq_len, **kwargs):
        super(Time2Vector, self).__init__()
        self.seq_len = seq_len

    def build(self, input_shape):
        self.weights_linear = self.add_weight(name='weight_linear',
                                    shape=(int(self.seq_len),),
                                    initializer='uniform',
                                    trainable=True)

        self.bias_linear = self.add_weight(name='bias_linear',
                                    shape=(int(self.seq_len),),
                                    initializer='uniform',
                                    trainable=True)

        self.weights_periodic = self.add_weight(name='weight_periodic',
                                    shape=(int(self.seq_len),),
                                    initializer='uniform',
                                    trainable=True)

        self.bias_periodic = self.add_weight(name='bias_periodic',
                                    shape=(int(self.seq_len),),
                                    initializer='uniform',
                                    trainable=True)

    def call(self, x):
        x = tf.math.reduce_mean(x[:,:,:], axis=-1) # Convert (batch, seq_len, 5) to (batch, seq_len)
        time_linear = self.weights_linear * x + self.bias_linear
        time_linear = tf.expand_dims(time_linear, axis=-1) # (batch, seq_len, 1)
        time_periodic = tf.math.sin(tf.multiply(x, self.weights_periodic) + self.bias_periodic)
        time_periodic = tf.expand_dims(time_periodic, axis=-1) # (batch, seq_len, 1)
        return tf.concat([time_linear, time_periodic], axis=-1) # (batch, seq_len, 2

In [8]:
class TransformerEncoder(layers.Layer):
    def __init__(self, embed_dim, num_heads, feed_forward_dim, rate=0.1):
        super().__init__()
        self.attn = layers.MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim)
        self.ffn = keras.Sequential(
            [
                layers.Dense(feed_forward_dim, activation="relu"),
                layers.Dense(embed_dim),
            ]
        )
        self.layernorm1 = layers.LayerNormalization(epsilon=1e-6)
        self.layernorm2 = layers.LayerNormalization(epsilon=1e-6)
        self.dropout1 = layers.Dropout(rate)
        self.dropout2 = layers.Dropout(rate)

    def call(self, inputs, training):
        attn_output = self.attn(inputs, inputs)
        attn_output = self.dropout1(attn_output, training=training)
        out1 = self.layernorm1(inputs + attn_output)
        ffn_output = self.ffn(out1)
        ffn_output = self.dropout2(ffn_output, training=training)
        return self.layernorm2(out1 + ffn_output)


class Transformer(keras.Model):
    def __init__(
            self,
            num_hid=64, # embed_dim - num of features
            time_steps=7,
            num_head = 2,
            num_feed_forward=128, # pointwise dim
            num_layers_enc = 4,
            time_embedding = False,
    ):
        super().__init__()
        self.num_hid = num_hid
        if time_embedding:
            self.num_hid += 2
            self.tv = Time2Vector(time_steps)
        else:
            self.tv = None
        self.numlayers_enc = num_layers_enc
        self.enc_input = layers.Input((time_steps, self.num_hid))
        self.encoder = keras.Sequential(
            [self.enc_input]
            + [
                TransformerEncoder(self.num_hid, num_head, num_feed_forward)
                for _ in range(num_layers_enc)
            ]
        )
        self.GlobalAveragePooling1D = layers.GlobalAveragePooling1D(data_format='channels_last')
        self.out = layers.Dense(units=1, activation='linear')        
        self.concat = tf.keras.layers.Concatenate(axis=-1)
        
    def call(self, inputs):
        if self.tv:
            x = self.tv(inputs)
            x = self.concat([inputs, x])
            x = self.encoder(x)
        else:
            x = self.encoder(inputs)
        x = self.GlobalAveragePooling1D(x)
        y = self.out(x)
        return y

In [9]:
seed = 47
TIMESTEPS = 1
warnings.filterwarnings("ignore")

# Reading the dataset

In [10]:
train_df = pd.read_csv("/kaggle/input/tabular-playground-series-mar-2022/train.csv", sep=',')


# Preprocessing


In [11]:
one_hot_encoder, train_df = label_encoder(train_df, 'direction')
train_df = preprocess_dates(train_df)
train_df = preprocess_holidays(train_df)
train_df = preprocess_timeseries(train_df)
x_train = train_df.drop(['row_id', 'time', 'congestion'], axis=1)
y_train = train_df['congestion']
x_train, x_test, y_train, y_test = train_test_split(x_train, y_train, test_size=0.2, random_state=seed, shuffle=False)

# Convert the data to 3D array shape

In [12]:
x_train = np.append(x_train, y_train.values.reshape(-1, 1), axis=1)
x_test = np.append(x_test, y_test.values.reshape(-1, 1), axis=1)
x_train, y_train = split_sequences(x_train, TIMESTEPS)
x_test, y_test = split_sequences(x_test, TIMESTEPS)

# Transformer with TIME2VEC

In [13]:
num_heads=2
num_layers_enc=2
num_feed_forward=64
num_features = x_train.shape[-1]
time_steps = TIMESTEPS
epochs = 100
batch_size = 128

model = Transformer(num_hid=num_features,
                        time_steps=time_steps,
                        time_embedding=True,
                        num_head=num_heads,
                        num_layers_enc=num_layers_enc,
                        num_feed_forward=num_feed_forward)

opt = tf.keras.optimizers.Adam()
loss = tf.keras.losses.MeanAbsoluteError()
model.compile(optimizer=opt, loss=loss)
model.fit(x_train, y_train, epochs=epochs, batch_size=batch_size, verbose=0)
print()
results = model.evaluate(x_test, y_test)
print(results)

2022-03-20 15:41:40.570003: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.
2022-03-20 15:41:41.210422: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)



5306/5306 [==============================] - 16s 3ms/step - loss: 8.1444
8.144418716430664


# Submission

In [14]:
del train_df, x_train, y_train, x_test, y_test
gc.collect()

2982

In [15]:
test_df = pd.read_csv("/kaggle/input/tabular-playground-series-mar-2022/test.csv", sep=',')

In [16]:
_, test_df = label_encoder(test_df, 'direction', one_hot_encoder)
test_df = preprocess_dates(test_df)
test_df = preprocess_holidays(test_df)
test_df = preprocess_timeseries(test_df)
x_test = test_df.drop(['row_id', 'time'], axis=1)
x_test = np.append(x_test, np.ones((x_test.shape[0], 1)), axis=1)
x_test, _ = split_sequences(x_test, TIMESTEPS)

In [17]:
target = model.predict(x_test).squeeze()
row_id =  test_df['row_id'].values
submission = pd.DataFrame({'row_id' : row_id, 'congestion' : target})

In [18]:
submission.head()

,row_id,congestion
0,848835,49.407795
1,848836,41.479584
2,848837,53.653522
3,848838,25.798189
4,848839,72.073982


In [19]:
submission.to_csv('submission.csv', index=False)